In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Add, Activation, Conv1D, Conv2D, Flatten
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
import numpy as np

In [23]:
# load the model
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data(path="mnist.npz")


In [24]:
# Preprocessing dei dati

train_images = train_images.reshape(60000,28,28,1)
test_images = test_images.reshape(10000,28,28,1)

# normalizzazione
train_images = (train_images / 255)
test_images = (test_images / 255)

# appiattimento di ogni immagine da un array 28x28 ad un array di 784 pixel
# train_images = train_images.reshape(-1, 28*28) # 784
# test_images = test_images.reshape(-1, 28*28)

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [28]:
shape_value = train_images.shape[0] # 28x28 -> 784
print(shape_value)

input = Input(shape=(28,28,1)) # posso creare anche custom objects che userò successivamente senza ripetere tutto.

conv_0 = Conv2D(
        64, 
        kernel_size=3, 
        activation='swish', 
        input_shape=(28,28,1) # This is the shape of each input image, 28,28,1 with the 1 signifying that the images are greyscale.
    )(input)

dense_0 = Dense(units=64, activation='relu')(conv_0)

residual_0 = Add()([conv_0, dense_0])
residual_0 = Activation('relu')(residual_0) # torna indietro con la derivata prima parziale della funzione relu

flatten_0 = Flatten()(residual_0)

output = Dense(
        units=10, 
        activation='softmax' # Softmax makes the output sum up to 1 so the output can be interpreted as probabilities.
    )(flatten_0)


model = tf.keras.models.Model(
    inputs=[input],
    outputs=[output],
    name="Conv-Test"
)

model.summary()


60000


ValueError: Inputs have incompatible shapes. Received shapes (26, 26, 64) and (26, 26, 16)

In [26]:
model.compile(
    optimizer=Adam(learning_rate=0.005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [27]:
history_res = model.fit(
    x=train_images,
    y=train_labels,
    epochs=50,
    batch_size=512,
    validation_data=(
        test_images,
        test_labels
    )
)

Epoch 1/50


2024-01-13 15:55:10.717826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


118/118 [==============================] - 4s 30ms/step - loss: 0.3772 - accuracy: 0.8888 - val_loss: 0.2807 - val_accuracy: 0.9225
Epoch 2/50
118/118 [==============================] - 3s 29ms/step - loss: 0.2621 - accuracy: 0.9264 - val_loss: 0.2272 - val_accuracy: 0.9350
Epoch 3/50
118/118 [==============================] - 3s 29ms/step - loss: 0.1659 - accuracy: 0.9533 - val_loss: 0.1431 - val_accuracy: 0.9583
Epoch 4/50
118/118 [==============================] - 3s 29ms/step - loss: 0.1147 - accuracy: 0.9671 - val_loss: 0.1120 - val_accuracy: 0.9658
Epoch 5/50
118/118 [==============================] - 3s 28ms/step - loss: 0.0870 - accuracy: 0.9746 - val_loss: 0.0917 - val_accuracy: 0.9722
Epoch 6/50
118/118 [==============================] - 3s 28ms/step - loss: 0.0702 - accuracy: 0.9797 - val_loss: 0.0897 - val_accuracy: 0.9716
Epoch 7/50
118/118 [==============================] - 3s 29ms/step - loss: 0.0584 - accuracy: 0.9819 - val_loss: 0.0872 - val_accuracy: 0.9741
Epoch 8/50

In [ ]:
# Stampe i risultati

# errore modello
plt.plot(history_res.history['loss'], color='red', label='train')
plt.plot(history_res.history['val_loss'], color='blue', label='test')
plt.title('Errore ResNet' )
plt.legend()
plt.show()

# Accuratezza del modello
plt.plot(history_res.history['accuracy'], color='red', label='train')
plt.plot(history_res.history['val_accuracy'], color='blue', label='test')
plt.title("Accuratezza ResNet")
plt.legend()
plt.show()

In [ ]:
evaluations_res = model.evaluate(
    test_images,
    test_labels
)

for i in range(0, len(model.metrics_names)):
    print(f"{model.metrics_names[i]} -> {evaluations_res[i]}")